In [4]:
%load_ext wurlitzer


import pickle
import trimesh as tm
import pyrender
import numpy as np
from visibility_new.visibility import Visibility

from joblib import Parallel,delayed
from tqdm import tqdm
from scipy.sparse import lil_matrix
from klampt import WorldModel,Geometry3D
from klampt import vis
from klampt.math import so3,se3
import trimesh
import os
import random
import numpy as np
import math
from klampt.model.create import moving_base_robot
from klampt.io import resource
from klampt.math import vectorops,so3
from klampt.model.collide import WorldCollider
from klampt.model import ik
import time
import numpy as np
from tqdm import tqdm
from klampt import IKObjective
import networkx as nx
from klampt.model.trajectory import RobotTrajectory,Trajectory
from klampt.model.create import *
from klampt import vis
from klampt.robotsim import Geometry3D,WorldModel
from klampt.math import se3,so3,vectorops
from klampt.vis import colorize
import sys
import time
from klampt.model import trajectory



mesh_file = './full_detail_hospital_cad_meters_fixed.obj'
pcd_file = "towerbot_robot_used_points_hospital.pcd"
reachable_points_file = 'towerbot_reachable.p'
irradiance_file = 'towerbot_irradiance_matrix_35_divs.p'
sampling_places_file = 'towerbot_sampling_places_35_divs.p'
configs_file = 'configs.p'
robot_file = "./primitives/towerbot.rob"
solutions_file = 'towerbot_solutions.p'
adjacency_file = 'towerbot_adj.p'
roadmap_file = 'towerbot_roadmap.p'
node_coords_file = 'towerbot_node_coords.p'
movie_dir = '../media/movies/towerbot/30_min/{:06d}.png'
tour_file = 'Towerbot_currTSP.txt'
skip = 1
dt = 10
max_step = 20
scale = 1
linknum = 2

def generate_video(mesh_file = './full_detail_hospital_cad_meters.obj',
                   pcd_file = "full_detail_robot_used_points_hospital.pcd",
                   reachable_points_file = 'full_detail_reachable.p',
                   irradiance_file = 'robot_clean_irradiance_matrix_35_divs.p',
                   sampling_places_file = 'armbot_sampling_places_35_divs.p',
                   configs_file = 'configs.p',
                   robot_file = "./primitives/disinfection_robot.rob",
                   solutions_file = 'robot_solutions.p',
                   adjacency_file = 'adj.p',
                   roadmap_file = 'roadmap.p',
                   node_coords_file = 'node_coords.p',
                   movie_dir = '../media/movies/armbot/30_min/s',
                   tour_file = 'currTSP.txt',
                   skip = 1,
                   dt = 0.4,
                   max_step = 20,
                   scale = 4,
                   linknum = 11, power = 80):


    irradiance_matrix = pickle.load(open(irradiance_file,'rb'))
    reachable = pickle.load(open(reachable_points_file,'rb'))
    #     configs = pickle.load(open(configs_file,'rb'))
    sampling_places = pickle.load(open(sampling_places_file,'rb'))


    #     sampling_places = sampling_places[reachable]
    #     configs = np.array(configs)[reachable].tolist()
    mu_single = 60*60*power*0.5*irradiance_matrix
    print("\n\n number of reachable points : {} \n".format(np.sum(reachable)))




    #process irradiation spots
    irradiance_matrix = pickle.load(open(irradiance_file,'rb'))
    vis_tester =Visibility(mesh_file,res = 512, useShader = True,createWnd = True)
    m = vis_tester.mesh
    areas = m.area()
    visible_points = np.asarray(((irradiance_matrix.sum(axis = 0)) > 0)).flatten()
    visible_areas = areas[visible_points]

    total_visible_areas = visible_areas.sum()
    total_visible_areas


    # irradiance_matrix = irradiance_matrix[reachable]

    visible_points = np.asarray(((irradiance_matrix.sum(axis = 0)) > 0)).flatten()

    visible_irradiance = irradiance_matrix[:,visible_points]

    areas = m.area()
    visible_areas = areas[visible_points]
    # we then define the visible area weights:



    visible_area_weights = np.exp(100*(visible_areas/visible_areas.sum()))
    visible_area_weights[:] = 1


    solutions = pickle.load(open(solutions_file,'rb'))
    visible_points = np.asarray(((irradiance_matrix.sum(axis = 0)) > 0)).flatten()
    visible_irradiance = irradiance_matrix[:,visible_points]



    # we now calculate the euclidean distance matrices for all the non-zero points:
    times = solutions[0]
    total_time = times.sum()
    points_mask = times>0.5
    used_points = sampling_places[points_mask,:]
    print(used_points.shape)
    mus = power*np.array(np.matmul(visible_irradiance.transpose(),times)).flatten()
    coverage = visible_areas[mus > 280].sum()/324.3
    print('Total Environment Coverage = {:.2f}%'.format(100*coverage))

    adjacency_matrix = pickle.load(open(adjacency_file,'rb'))
    roadmap = pickle.load(open(roadmap_file,'rb'))
    node_coords = pickle.load(open(node_coords_file,'rb'))

    from getToursAndPaths import getTour, readTourFile, getPathFromPrm, getFinalPath

    distances = np.zeros(shape = (adjacency_matrix.shape[0]+1,adjacency_matrix.shape[0]+1))
    distances[1:,1:] = 100*adjacency_matrix.copy()
    #     getTour(distances, 'currTSP') # We just have an arbitrary name since it doesn't matter - can change this so that user can input filename if desired
    tour = readTourFile(tour_file, used_points)
    tour = (np.array(tour[1:])-1).tolist()
    indices = np.array(range(sampling_places.shape[0]))
    used_indices = indices[points_mask]
    tour_indices = used_indices[tour]

    # We then calculate the total distance travelled - and compute final trajectory:
    total_distance = 0 
    final_trajectory = []
    for i in range(len(tour)-1):
        current_point = tour[i]
        next_point = tour[i+1]
        total_distance += adjacency_matrix[current_point,next_point]
        nodes_path = nx.algorithms.shortest_path(roadmap,source = current_point,target = next_point,weight = 'weight')
    #     print(nodes_path)
    #     print(nodes_path)
        traj = [node_coords[j][:12] for j in nodes_path]
    #     print(trajectory)
        final_trajectory.append(traj[:-1])
    # we then add the final point to the trajectory
    final_trajectory.append([traj[-1]])
    #     final_trajectory.append(pathDict[current_point,next_point])
    print("Total Distance Travelled by the EE = {} | Total Time Spent in Transit = {} ".format(total_distance,total_distance/30))






    #prepping animations
    time_allowances = np.linspace(0.00,0.5,50)
    times = solutions[0]
    partial_times  = []
    for allowance in time_allowances:
        partial_time = np.zeros(sampling_places.shape[0])

        for k in range(len(tour_indices)-1):
            total_time = times[tour_indices[:k+1]].sum()/(3600)
            if(total_time >= allowance):
                k = k-1
                break
        partial_time[tour_indices[:k+1]] = times[tour_indices[:k+1]]
        print(partial_time.sum()/3600)
        partial_times.append(partial_time)

    # trajectory_times:

    total_times = []
    for k in range(len(tour_indices)):
        partial_time = np.zeros(sampling_places.shape[0])
        partial_time[tour_indices[:k+1]] = times[tour_indices[:k+1]]
        total_times.append(partial_time.sum()/3600)
    #     print(total_times[-1])
        partial_times.append(partial_time)
    total_times = np.array(total_times)

    # we then determine the strategic stops
    first = np.argmin(np.abs(total_times-0.125))
    second = np.argmin(np.abs(total_times-0.25))
    third =  np.argmin(np.abs(total_times-0.375))
    fourth = total_times.shape[0] - 1



    stop_spots = [first,second,third,fourth]
    timed_trajectory = []
    trajectory_times = []
    prev_time = 0
    for index,path in enumerate(final_trajectory):
        section_length = len(path)
        section_times = np.zeros(section_length) + prev_time
        section_times = section_times + np.arange(section_length)*((total_times[index]-prev_time)/section_length)
        section_times = section_times*3600
        prev_time = total_times[index]
        timed_trajectory.extend(path)
        trajectory_times.extend(section_times)

    # setting up basic animations


    vis.init('PyQt')
    # del robot
    a = Geometry3D()
    res = a.loadFile(mesh_file)

    b = Geometry3D()
    res = b.loadFile(pcd_file)
    if(not res):
        print('failed loading geometry!')
    w = WorldModel()

    w.loadElement(robot_file)
    # w,robot,lamp,collider = setup_robot_and_light(mesh_file = mesh_file)
    robot = w.robot(0)
    robot.link(3).appearance().setColor(127/255,0/255,255/255,1)

    cfig = robot.getConfig()
    cfig[2] = 0.08
    robot.setConfig(cfig)
    w.makeRigidObject("a")
    w.rigidObject(0).geometry().set(a)
    w.makeRigidObject("b")
    w.rigidObject(1).geometry().set(b)

    final_timed_trajectory = np.zeros(shape = (len(timed_trajectory),4))
    final_timed_trajectory[:,2] = 0.08
    final_timed_trajectory[:,3] = 0
    final_timed_trajectory[:,:2] = np.array(timed_trajectory)


    robotTraj = RobotTrajectory(robot,milestones = final_timed_trajectory,times = trajectory_times).discretize(dt)
    linkTraj = robotTraj.getLinkTrajectory(linknum,dt) #.getPositionTrajectory([0,0,0])

    times = solutions[0]
    final_times = []
    for this_time in robotTraj.times:
        for j in range(len(tour_indices)):
            partial_time = np.zeros(sampling_places.shape[0])
            partial_time[tour_indices[:j+1]] = times[tour_indices[:j+1]]
            total_time = partial_time.sum()
    #         print(this_time,total_time)
            if(total_time >= this_time):
                break
        partial_time[tour_indices[j]] = this_time-times[tour_indices[:j]].sum()
        final_times.append(partial_time)
    final_times[-1] = solutions[0]
    final_times = np.array(final_times)
    # robot,w
    value = None
    cmap = None
    feature = None
    #     time.sleep(20*np.random.rand())
    vis.add("A",a)
    # vis.add("B",b,pointSize = 15)
    vis.add("robot",robot)
    curr_milestones = sampling_places[tour_indices[:1],:]
    full_trajectory = []
    for i in final_trajectory:
        full_trajectory.extend(i)
    traj = trajectory.Trajectory(milestones= full_trajectory)#curr_milestones.tolist())
    # vis.add("traj",traj)
    # we then calculate the percentage of disinfection fluence:
    # disinfected_vecs = []
    partial_times_array = np.array(partial_times).transpose()
    # for times in partial_times:
    # mu = 80*np.matmul(visible_irradiance.transpose().toarray(),final_ties)
    # disinfected_vecs = np.clip(mu/(1.1*280),0,1).transpose()

    # we then add all zeros to begin the illustration
    # tmp = [np.zeros(disinfected_vecs[0].shape)]
    # tmp[0][0] = 1
    # disinfected_vecs_new = disinfected_vecs.copy()
    vis.visualization.setBackgroundColor(1, 1, 1, a=1)

    vis.show()

    # eliminating draw distance
    vis.lock()
    # time.sleep(0.5)
    vp = vis.getViewport()
    # vp.h = 640
    # vp.w = 640
    vp.clippingplanes = [0.1,10000]
    tform = pickle.load(open('tform.p','rb'))
    vp.setTransform(tform)
    vp.w = 1853//scale
    vp.h = 1123//scale

    # vis.setViewport(vp)
    vis.scene().setViewport(vp)
    vis.unlock()
    new_partial_times = final_times
    max_t = 0
    for i in partial_times:
        max_t = max(max_t,np.max(i))


        #animation steps
    tholds = [0.25,0.5,0.75,0.999995]
    thold_index = 0
    thold = tholds[thold_index]
    # i = 56
    # while vis.shown():
    print('gets here?')
    done = get_done_set(movie_dir)
    total_frames = set(range(len(robotTraj.milestones)))
    to_do = total_frames - done 
    for i in range(len(to_do)):

    #     time.sleep((1/120)*solutions[0][tour_indices[i]])
    #     i = i%(len(disinfected_vecs_new))
        times = new_partial_times[i]
        disinfection_fractions = areas.copy()
        disinfection_fractions[:] = 0
        mu = 80*np.matmul(visible_irradiance.transpose(),times.transpose())
        disinfected_vecs = np.clip(mu/(1.1*280),0,1).transpose()
        if(i == 0):
            disinfected_vecs[0] = 10
    #         print(disinfected_vecs.sum())
        #     disinfected_vecs[0] = 0.0001
    #     if(times.sum()/total_time >= thold):
    #         print('\nstopping at {} % due to total disinfection time {}\n'.format(thold,times.sum()/total_time))
    #         thold_index = (thold_index+1)%4
    #         thold = tholds[thold_index]
    #     if(i == 57):
    #         break
    #     time.sleep(0.2)
        vis.lock()
        fraction = areas[visible_points][disinfected_vecs >= 1].sum()/250.2
        vis.addText('disinfection_progress','Reachable Coverage = {:.1f} %'.format(100*fraction),position = [10//scale,10//scale],size = 36//scale)
        vis.addText('disinfection_time','Time = {:.2f} minutes'.format(times.sum()/60),position = [10//scale,50//scale],size = 36//scale)

    #     traj = trajectory.Trajectory(milestones=so_far)

        disinfection_fractions[visible_points] = disinfected_vecs




        a_app = colorize.colorize(w.rigidObject(0),disinfection_fractions,'RdYlGn','faces',lighting=[0,0,-1])

        robot.setConfig(robotTraj.milestones[i])
#         vis.remove("A")
        vis.add("A",a_app)
        traj = Trajectory(milestones = linkTraj.getPositionTrajectory([0,0,0]).milestones[:i])
        if(i!= 0 ):
            vis.add("traj",traj)
        vis.unlock()
        time.sleep(0.25)
        vis.threadCall(lambda :vis.scene().program.save_screen(movie_dir.format(i)))
#         vis.scene().program.save_screen(movie_dir.format(i))
    
    vis.kill()

def get_done_set(movie_dir):
    from glob import glob
    import pandas as pd
    a = glob(movie_dir)
    if(a):
        a = pd.Series(a)
        b = a.str.split('/',expand = True).iloc[:,-1]
        c = b.str.split('.', expand = True).iloc[:,0].astype(int)
        d = set(c)
        return d
    else:
        return set({})

The wurlitzer extension is already loaded. To reload it, use:
  %reload_ext wurlitzer


In [5]:
generate_video(mesh_file = './remeshed_hospital_room_full.obj',
               pcd_file = "towerbot_robot_used_points_hospital.pcd",
               reachable_points_file = 'towerbot_reachable.p',
               irradiance_file = 'towerbot_irradiance_matrix_35_divs.p',
               sampling_places_file = 'towerbot_sampling_places_35_divs.p',
               configs_file = 'configs.p',
               robot_file = "./primitives/towerbot.rob",
               solutions_file = 'towerbot_solutions.p',
               adjacency_file = 'towerbot_adj.p',
               roadmap_file = 'towerbot_roadmap.p',
               node_coords_file = 'towerbot_node_coords.p',
               movie_dir = '../media/movies/towerbot/30_min/{:06d}.png',
               tour_file = 'Towerbot_currTSP.txt',
               linknum = 2,scale = 1)




 number of reachable points : 362 

(110, 2)
Total Environment Coverage = 59.31%
Total Distance Travelled by the EE = 41.59072409555094 | Total Time Spent in Transit = 1.386357469851698 
0.0
0.008967077385403966
0.018141478673765823
0.018141478673765823
0.039858838065872135
0.0479031757628119
0.0605519155740601
0.06179495095275322
0.08102350025634764
0.08778134208388964
0.1015767445689556
0.10859612336638046
0.12140902301692992
0.13265264467276647
0.14046740758864482
0.14046740758864482
0.15492060184794665
0.16395723797202177
0.18141045131716016
0.1900054021880167
0.20033054630149416
0.2096347606443962
0.2143083906999864
0.2143083906999864
0.2393694918603556
0.2497453938699443
0.2640187420172224
0.27427571731391864
0.2851696329600629
0.29391364622940047
0.2993079733328562
0.3141143822270932
0.31982429728061706
0.32710648707103096
0.32710648707103096
0.35259612237470944
0.36714094753574245
0.37044506032807073
0.3855499749631013
0.39052085888855653
0.39052085888855653
0.415794671416342

######### QGLWidget Initialize GL ###############
TriMeshTopology: mesh has 11912 triangles with duplicate neighbors!
  Triangle range 55 to 128224
  May see strange results for some triangle mesh operations
TriMeshTopology: mesh has 11912 triangles with duplicate neighbors!
  Triangle range 55 to 128224
  May see strange results for some triangle mesh operations
Saving screen to ../media/movies/towerbot/30_min/000001.png
TriMeshTopology: mesh has 11912 triangles with duplicate neighbors!
  Triangle range 55 to 128224
  May see strange results for some triangle mesh operations
Saving screen to ../media/movies/towerbot/30_min/000003.png
Saving screen to ../media/movies/towerbot/30_min/000003.png
TriMeshTopology: mesh has 11912 triangles with duplicate neighbors!
  Triangle range 55 to 128224
  May see strange results for some triangle mesh operations
Saving screen to ../media/movies/towerbot/30_min/000004.png
TriMeshTopology: mesh has 11912 triangles with duplicate neighbors!
  Triangle

TriMeshTopology: mesh has 11912 triangles with duplicate neighbors!
  Triangle range 55 to 128224
  May see strange results for some triangle mesh operations
TriMeshTopology: mesh has 11912 triangles with duplicate neighbors!
  Triangle range 55 to 128224
  May see strange results for some triangle mesh operations
Saving screen to ../media/movies/towerbot/30_min/000039.png
TriMeshTopology: mesh has 11912 triangles with duplicate neighbors!
  Triangle range 55 to 128224
  May see strange results for some triangle mesh operations
Saving screen to ../media/movies/towerbot/30_min/000041.png
Saving screen to ../media/movies/towerbot/30_min/000041.png
TriMeshTopology: mesh has 11912 triangles with duplicate neighbors!
  Triangle range 55 to 128224
  May see strange results for some triangle mesh operations
Saving screen to ../media/movies/towerbot/30_min/000042.png
TriMeshTopology: mesh has 11912 triangles with duplicate neighbors!
  Triangle range 55 to 128224
  May see strange results for 

TriMeshTopology: mesh has 11912 triangles with duplicate neighbors!
  Triangle range 55 to 128224
  May see strange results for some triangle mesh operations
Saving screen to ../media/movies/towerbot/30_min/000077.png
TriMeshTopology: mesh has 11912 triangles with duplicate neighbors!
  Triangle range 55 to 128224
  May see strange results for some triangle mesh operations
Saving screen to ../media/movies/towerbot/30_min/000078.png
TriMeshTopology: mesh has 11912 triangles with duplicate neighbors!
  Triangle range 55 to 128224
  May see strange results for some triangle mesh operations
Saving screen to ../media/movies/towerbot/30_min/000079.png
TriMeshTopology: mesh has 11912 triangles with duplicate neighbors!
  Triangle range 55 to 128224
  May see strange results for some triangle mesh operations
Saving screen to ../media/movies/towerbot/30_min/000080.png
TriMeshTopology: mesh has 11912 triangles with duplicate neighbors!
  Triangle range 55 to 128224
  May see strange results for 

TriMeshTopology: mesh has 11912 triangles with duplicate neighbors!
  Triangle range 55 to 128224
  May see strange results for some triangle mesh operations
Saving screen to ../media/movies/towerbot/30_min/000115.png
TriMeshTopology: mesh has 11912 triangles with duplicate neighbors!
  Triangle range 55 to 128224
  May see strange results for some triangle mesh operations
Saving screen to ../media/movies/towerbot/30_min/000116.png
TriMeshTopology: mesh has 11912 triangles with duplicate neighbors!
  Triangle range 55 to 128224
  May see strange results for some triangle mesh operations
Saving screen to ../media/movies/towerbot/30_min/000117.png
TriMeshTopology: mesh has 11912 triangles with duplicate neighbors!
  Triangle range 55 to 128224
  May see strange results for some triangle mesh operations
Saving screen to ../media/movies/towerbot/30_min/000118.png
TriMeshTopology: mesh has 11912 triangles with duplicate neighbors!
  Triangle range 55 to 128224
  May see strange results for 

...

Saving screen to ../media/movies/towerbot/30_min/004257.png
Saving screen to ../media/movies/towerbot/30_min/004257.png
TriMeshTopology: mesh has 11912 triangles with duplicate neighbors!
  Triangle range 55 to 128224
  May see strange results for some triangle mesh operations
Saving screen to ../media/movies/towerbot/30_min/004258.png
TriMeshTopology: mesh has 11912 triangles with duplicate neighbors!
  Triangle range 55 to 128224
  May see strange results for some triangle mesh operations
Saving screen to ../media/movies/towerbot/30_min/004259.png
TriMeshTopology: mesh has 11912 triangles with duplicate neighbors!
  Triangle range 55 to 128224
  May see strange results for some triangle mesh operations
Saving screen to ../media/movies/towerbot/30_min/004260.png
TriMeshTopology: mesh has 11912 triangles with duplicate neighbors!
  Triangle range 55 to 128224
  May see strange results for some triangle mesh operations
Saving screen to ../media/movies/towerbot/30_min/004261.png
TriMeshT

Saving screen to ../media/movies/towerbot/30_min/004295.png
TriMeshTopology: mesh has 11912 triangles with duplicate neighbors!
  Triangle range 55 to 128224
  May see strange results for some triangle mesh operations
Saving screen to ../media/movies/towerbot/30_min/004296.png
TriMeshTopology: mesh has 11912 triangles with duplicate neighbors!
  Triangle range 55 to 128224
  May see strange results for some triangle mesh operations
Saving screen to ../media/movies/towerbot/30_min/004297.png
TriMeshTopology: mesh has 11912 triangles with duplicate neighbors!
  Triangle range 55 to 128224
  May see strange results for some triangle mesh operations
TriMeshTopology: mesh has 11912 triangles with duplicate neighbors!
  Triangle range 55 to 128224
  May see strange results for some triangle mesh operations
Saving screen to ../media/movies/towerbot/30_min/004298.png
TriMeshTopology: mesh has 11912 triangles with duplicate neighbors!
  Triangle range 55 to 128224
  May see strange results for 

Saving screen to ../media/movies/towerbot/30_min/004333.png
TriMeshTopology: mesh has 11912 triangles with duplicate neighbors!
  Triangle range 55 to 128224
  May see strange results for some triangle mesh operations
Saving screen to ../media/movies/towerbot/30_min/004334.png
TriMeshTopology: mesh has 11912 triangles with duplicate neighbors!
  Triangle range 55 to 128224
  May see strange results for some triangle mesh operations
Saving screen to ../media/movies/towerbot/30_min/004335.png
TriMeshTopology: mesh has 11912 triangles with duplicate neighbors!
  Triangle range 55 to 128224
  May see strange results for some triangle mesh operations
Saving screen to ../media/movies/towerbot/30_min/004336.png
TriMeshTopology: mesh has 11912 triangles with duplicate neighbors!
  Triangle range 55 to 128224
  May see strange results for some triangle mesh operations
Saving screen to ../media/movies/towerbot/30_min/004337.png
TriMeshTopology: mesh has 11912 triangles with duplicate neighbors!


Saving screen to ../media/movies/towerbot/30_min/004371.png
TriMeshTopology: mesh has 11912 triangles with duplicate neighbors!
  Triangle range 55 to 128224
  May see strange results for some triangle mesh operations
Saving screen to ../media/movies/towerbot/30_min/004372.png
TriMeshTopology: mesh has 11912 triangles with duplicate neighbors!
  Triangle range 55 to 128224
  May see strange results for some triangle mesh operations
Saving screen to ../media/movies/towerbot/30_min/004373.png
TriMeshTopology: mesh has 11912 triangles with duplicate neighbors!
  Triangle range 55 to 128224
  May see strange results for some triangle mesh operations
Saving screen to ../media/movies/towerbot/30_min/004374.png
TriMeshTopology: mesh has 11912 triangles with duplicate neighbors!
  Triangle range 55 to 128224
  May see strange results for some triangle mesh operations
Saving screen to ../media/movies/towerbot/30_min/004375.png
TriMeshTopology: mesh has 11912 triangles with duplicate neighbors!


TriMeshTopology: mesh has 11912 triangles with duplicate neighbors!
  Triangle range 55 to 128224
  May see strange results for some triangle mesh operations
Saving screen to ../media/movies/towerbot/30_min/004409.png
Saving screen to ../media/movies/towerbot/30_min/004410.png
TriMeshTopology: mesh has 11912 triangles with duplicate neighbors!
  Triangle range 55 to 128224
  May see strange results for some triangle mesh operations
TriMeshTopology: mesh has 11912 triangles with duplicate neighbors!
  Triangle range 55 to 128224
  May see strange results for some triangle mesh operations
Saving screen to ../media/movies/towerbot/30_min/004411.png
Saving screen to ../media/movies/towerbot/30_min/004412.png
TriMeshTopology: mesh has 11912 triangles with duplicate neighbors!
  Triangle range 55 to 128224
  May see strange results for some triangle mesh operations
Saving screen to ../media/movies/towerbot/30_min/004413.png
Saving screen to ../media/movies/towerbot/30_min/004413.png
TriMeshT

In [ ]:
%load_ext wurlitzer


import pickle
import trimesh as tm
import pyrender
import numpy as np
import visibility
from importlib import reload
reload(visibility)
from visibility import Visibility, Mesh
from joblib import Parallel,delayed
from tqdm import tqdm
from scipy.sparse import lil_matrix
from klampt import WorldModel,Geometry3D
from klampt import vis
from klampt.math import so3,se3
import trimesh
import os
import random
import numpy as np
import math
from klampt.model.create import moving_base_robot
from klampt.io import resource
from klampt.math import vectorops,so3
from klampt.model.collide import WorldCollider
from klampt.model import ik
import time
import numpy as np
from tqdm import tqdm
from klampt import IKObjective
import networkx as nx
from klampt.model.trajectory import RobotTrajectory,Trajectory
from klampt.model.create import *
from klampt import vis
from klampt.robotsim import Geometry3D,WorldModel
from klampt.math import se3,so3,vectorops
from klampt.vis import colorize
import sys
import time
from klampt.model import trajectory



mesh_file = './full_detail_hospital_cad_meters_fixed.obj'
pcd_file = "towerbot_robot_used_points_hospital.pcd"
reachable_points_file = 'towerbot_reachable.p'
irradiance_file = 'towerbot_irradiance_matrix_35_divs.p'
sampling_places_file = 'towerbot_sampling_places_35_divs.p'
configs_file = 'configs.p'
robot_file = "./primitives/towerbot.rob"
solutions_file = 'towerbot_solutions.p'
adjacency_file = 'towerbot_adj.p'
roadmap_file = 'towerbot_roadmap.p'
node_coords_file = 'towerbot_node_coords.p'
movie_dir = '../media/movies/towerbot/30_min/{:06d}.png'
tour_file = 'Towerbot_currTSP.txt'
skip = 1
dt = 10
max_step = 20
scale = 1
linknum = 2

def get_solution_trajectory(mesh_file = './full_detail_hospital_cad_meters.obj',
                   pcd_file = "full_detail_robot_used_points_hospital.pcd",
                   reachable_points_file = 'full_detail_reachable.p',
                   irradiance_file = 'robot_clean_irradiance_matrix_35_divs.p',
                   sampling_places_file = 'armbot_sampling_places_35_divs.p',
                   configs_file = 'configs.p',
                   robot_file = "./primitives/disinfection_robot.rob",
                   solutions_file = 'robot_solutions.p',
                   adjacency_file = 'adj.p',
                   roadmap_file = 'roadmap.p',
                   node_coords_file = 'node_coords.p',
                   movie_dir = '../media/movies/armbot/30_min/{:06d}.png',
                   tour_file = 'currTSP.txt',
                   skip = 1,
                   dt = 0.4,
                   max_step = 20,
                   scale = 4,
                   linknum = 11,
                    power = 80):


    irradiance_matrix = pickle.load(open(irradiance_file,'rb'))
    reachable = pickle.load(open(reachable_points_file,'rb'))
    #     configs = pickle.load(open(configs_file,'rb'))
    sampling_places = pickle.load(open(sampling_places_file,'rb'))


    #     sampling_places = sampling_places[reachable]
    #     configs = np.array(configs)[reachable].tolist()
    mu_single = 60*60*80*0.5*irradiance_matrix
    print("\n\n number of reachable points : {} \n".format(np.sum(reachable)))




    #process irradiation spots
    irradiance_matrix = pickle.load(open(irradiance_file,'rb'))

    m=Mesh(mesh_file)
    areas = m.area()
    visible_points = np.asarray(((irradiance_matrix.sum(axis = 0)) > 0)).flatten()
    visible_areas = areas[visible_points]

    total_visible_areas = visible_areas.sum()
    total_visible_areas


    irradiance_matrix = irradiance_matrix[reachable]

    visible_points = np.asarray(((irradiance_matrix.sum(axis = 0)) > 0)).flatten()

    visible_irradiance = irradiance_matrix[:,visible_points]

    m=Mesh(mesh_file)
    areas = m.area()
    visible_areas = areas[visible_points]
    # we then define the visible area weights:



    visible_area_weights = np.exp(100*(visible_areas/visible_areas.sum()))
    visible_area_weights[:] = 1


    solutions = pickle.load(open(solutions_file,'rb'))
    visible_points = np.asarray(((irradiance_matrix.sum(axis = 0)) > 0)).flatten()
    visible_irradiance = irradiance_matrix[:,visible_points]



    # we now calculate the euclidean distance matrices for all the non-zero points:
    times = solutions[0]
    total_time = times.sum()
    points_mask = times>0.5
    used_points = sampling_places[points_mask,:]
    print(used_points.shape)
    mus = power*np.array(np.matmul(visible_irradiance.transpose(),times)).flatten()
    coverage = visible_areas[mus > 280].sum()/324.3
    print('Total Environment Coverage = {:.2f}%'.format(100*coverage))

    adjacency_matrix = pickle.load(open(adjacency_file,'rb'))
    roadmap = pickle.load(open(roadmap_file,'rb'))
    node_coords = pickle.load(open(node_coords_file,'rb'))

    from getToursAndPaths import getTour, readTourFile, getPathFromPrm, getFinalPath

    distances = np.zeros(shape = (adjacency_matrix.shape[0]+1,adjacency_matrix.shape[0]+1))
    distances[1:,1:] = 100*adjacency_matrix.copy()
    #     getTour(distances, 'currTSP') # We just have an arbitrary name since it doesn't matter - can change this so that user can input filename if desired
    tour = readTourFile(tour_file, used_points)
    tour = (np.array(tour[1:])-1).tolist()
    indices = np.array(range(sampling_places.shape[0]))
    used_indices = indices[points_mask]
    tour_indices = used_indices[tour]

    # We then calculate the total distance travelled - and compute final trajectory:
    total_distance = 0 
    final_trajectory = []
    for i in range(len(tour)-1):
        current_point = tour[i]
        next_point = tour[i+1]
        total_distance += adjacency_matrix[current_point,next_point]
        nodes_path = nx.algorithms.shortest_path(roadmap,source = current_point,target = next_point,weight = 'weight')
    #     print(nodes_path)
    #     print(nodes_path)
        traj = [node_coords[j][:12] for j in nodes_path]
    #     print(trajectory)
        final_trajectory.append(traj[:-1])
    # we then add the final point to the trajectory
    final_trajectory.append([traj[-1]])
    #     final_trajectory.append(pathDict[current_point,next_point])
    print("Total Distance Travelled by the EE = {} | Total Time Spent in Transit = {} ".format(total_distance,total_distance/30))






    #prepping animations
    time_allowances = np.linspace(0.00,0.5,50)
    times = solutions[0]
    partial_times  = []
    for allowance in time_allowances:
        partial_time = np.zeros(sampling_places.shape[0])

        for k in range(len(tour_indices)-1):
            total_time = times[tour_indices[:k+1]].sum()/(3600)
            if(total_time >= allowance):
                k = k-1
                break
        partial_time[tour_indices[:k+1]] = times[tour_indices[:k+1]]
        print(partial_time.sum()/3600)
        partial_times.append(partial_time)

    # trajectory_times:

    total_times = []
    for k in range(len(tour_indices)):
        partial_time = np.zeros(sampling_places.shape[0])
        partial_time[tour_indices[:k+1]] = times[tour_indices[:k+1]]
        total_times.append(partial_time.sum()/3600)
    #     print(total_times[-1])
        partial_times.append(partial_time)
    total_times = np.array(total_times)

    # we then determine the strategic stops
    first = np.argmin(np.abs(total_times-0.125))
    second = np.argmin(np.abs(total_times-0.25))
    third =  np.argmin(np.abs(total_times-0.375))
    fourth = total_times.shape[0] - 1



    stop_spots = [first,second,third,fourth]
    timed_trajectory = []
    trajectory_times = []
    prev_time = 0
    for index,path in enumerate(final_trajectory):
        section_length = len(path)
        section_times = np.zeros(section_length) + prev_time
        section_times = section_times + np.arange(section_length)*((total_times[index]-prev_time)/section_length)
        section_times = section_times*3600
        prev_time = total_times[index]
        timed_trajectory.extend(path)
        trajectory_times.extend(section_times)

    # setting up basic animations


    vis.init('PyQt')
    # del robot
    a = Geometry3D()
    res = a.loadFile(mesh_file)

    b = Geometry3D()
    res = b.loadFile(pcd_file)
    if(not res):
        print('failed loading geometry!')
    w = WorldModel()

    w.loadElement(robot_file)
    # w,robot,lamp,collider = setup_robot_and_light(mesh_file = mesh_file)
    robot = w.robot(0)
    robot.link(3).appearance().setColor(127/255,0/255,255/255,1)

    cfig = robot.getConfig()
    cfig[2] = 0.08
    robot.setConfig(cfig)
    w.makeRigidObject("a")
    w.rigidObject(0).geometry().set(a)
    w.makeRigidObject("b")
    w.rigidObject(1).geometry().set(b)

    final_timed_trajectory = np.zeros(shape = (len(timed_trajectory),4))
    final_timed_trajectory[:,2] = 0.08
    final_timed_trajectory[:,3] = 0
    final_timed_trajectory[:,:2] = np.array(timed_trajectory)


    robotTraj = RobotTrajectory(robot,milestones = final_timed_trajectory,times = trajectory_times).discretize(dt)
    linkTraj = robotTraj.getLinkTrajectory(linknum,dt) #.getPositionTrajectory([0,0,0])
    return linkTraj,robotTraj,final_timed_trajectory,robot
    times = solutions[0]
    final_times = []
    for this_time in robotTraj.times:
        for j in range(len(tour_indices)):
            partial_time = np.zeros(sampling_places.shape[0])
            partial_time[tour_indices[:j+1]] = times[tour_indices[:j+1]]
            total_time = partial_time.sum()
    #         print(this_time,total_time)
            if(total_time >= this_time):
                break
        partial_time[tour_indices[j]] = this_time-times[tour_indices[:j]].sum()
        final_times.append(partial_time)
    final_times[-1] = solutions[0]
    final_times = np.array(final_times)
    # robot,w
    value = None
    cmap = None
    feature = None
    #     time.sleep(20*np.random.rand())
    vis.add("A",a)
    # vis.add("B",b,pointSize = 15)
    vis.add("robot",robot)
    curr_milestones = sampling_places[tour_indices[:1],:]
    full_trajectory = []
    for i in final_trajectory:
        full_trajectory.extend(i)
    traj = trajectory.Trajectory(milestones= full_trajectory)#curr_milestones.tolist())
    # vis.add("traj",traj)
    # we then calculate the percentage of disinfection fluence:
    # disinfected_vecs = []
    partial_times_array = np.array(partial_times).transpose()
    # for times in partial_times:
    # mu = 80*np.matmul(visible_irradiance.transpose().toarray(),final_ties)
    # disinfected_vecs = np.clip(mu/(1.1*280),0,1).transpose()

    # we then add all zeros to begin the illustration
    # tmp = [np.zeros(disinfected_vecs[0].shape)]
    # tmp[0][0] = 1
    # disinfected_vecs_new = disinfected_vecs.copy()
    vis.visualization.setBackgroundColor(1, 1, 1, a=1)

    vis.show()

    # eliminating draw distance
    vis.lock()
    # time.sleep(0.5)
    vp = vis.getViewport()
    # vp.h = 640
    # vp.w = 640
    vp.clippingplanes = [0.1,10000]
    tform = pickle.load(open('tform.p','rb'))
    vp.setTransform(tform)
    vp.w = 1853//scale
    vp.h = 1123//scale

    # vis.setViewport(vp)
    vis.scene().setViewport(vp)
    vis.unlock()
    new_partial_times = final_times
    max_t = 0
    for i in partial_times:
        max_t = max(max_t,np.max(i))


        #animation steps
    tholds = [0.25,0.5,0.75,0.999995]
    thold_index = 0
    thold = tholds[thold_index]
    # i = 56
    # while vis.shown():
    for i in range(1508,len(robotTraj.milestones)):

    #     time.sleep((1/120)*solutions[0][tour_indices[i]])
    #     i = i%(len(disinfected_vecs_new))
        times = new_partial_times[i]
        disinfection_fractions = areas.copy()
        disinfection_fractions[:] = 0
        mu = 80*np.matmul(visible_irradiance.transpose(),times.transpose())
        disinfected_vecs = np.clip(mu/(1.1*280),0,1).transpose()
    #         print(disinfected_vecs.sum())
        #     disinfected_vecs[0] = 0.0001
    #     if(times.sum()/total_time >= thold):
    #         print('\nstopping at {} % due to total disinfection time {}\n'.format(thold,times.sum()/total_time))
    #         thold_index = (thold_index+1)%4
    #         thold = tholds[thold_index]
    #     if(i == 57):
    #         break
    #     time.sleep(0.2)
        fraction = areas[visible_points][disinfected_vecs >= 1].sum()/272
        vis.addText('disinfection_progress','Reachable Coverage = {:.1f} %'.format(100*fraction),position = [10//scale,10//scale],size = 36//scale)
        vis.addText('disinfection_time','Time = {:.2f} minutes'.format(times.sum()/60),position = [10//scale,50//scale],size = 36//scale)

    #     traj = trajectory.Trajectory(milestones=so_far)

        disinfection_fractions[visible_points] = disinfected_vecs




        a_app = colorize.colorize(w.rigidObject(0),disinfection_fractions,'RdYlGn','faces',lighting=[0,0,-1])
    #     b_app = colorize.colorize(w.rigidObject(1),times,'magma')
    #     b_app.appearance().setPointSize(10)
    #     curr_pos = sampling_places[tour_indices[i],:]
        vis.lock()
        robot.setConfig(robotTraj.milestones[i])
        if(i != 0):
            vis.remove("A")
            vis.add("A",a_app)
    #     vis.setAttribute("B","size",10)
    #     vis.remove("B")
    #     vis.add("B",b_app)
    #     vis.dirty("B")
    #     vis.remove("traj")
        traj = Trajectory(milestones = linkTraj.getPositionTrajectory([0,0,0]).milestones[:i])
        if(i!= 0 ):
            vis.add("traj",traj)
        vis.unlock()
        vis.threadCall(lambda :vis.scene().program.save_screen(movie_dir.format(i)))
    #     vis.scene().program.save_screen('{}.png'.format(i),multithreaded = False)
        print('\n {} \n '.format(i))
    #     break
    #         time.sleep(0.2)


    #     i +=1
    #     time.sleep(5)
    vis.kill()

In [ ]:
l,r,f,robot = get_solution_trajectory(mesh_file = './full_detail_hospital_cad_meters_fixed.obj',
               pcd_file = "towerbot_robot_used_points_hospital.pcd",
               reachable_points_file = 'towerbot_reachable.p',
               irradiance_file = 'towerbot_irradiance_matrix_35_divs.p',
               sampling_places_file = 'towerbot_sampling_places_35_divs.p',
               configs_file = 'configs.p',
               robot_file = "./primitives/towerbot.rob",
               solutions_file = 'towerbot_solutions.p',
               adjacency_file = 'towerbot_adj.p',
               roadmap_file = 'towerbot_roadmap.p',
               node_coords_file = 'towerbot_node_coords.p',
               movie_dir = '../media/movies/towerbot/30_min/{:06d}.png',
               tour_file = 'Towerbot_currTSP.txt',
               linknum = 2,scale = 1,power = 52)

In [ ]:
f

tj = trajectory.Trajectory(milestones= f, times = 1800*np.array(list(range(len(f))))/len(f))
# robotTraj = RobotTrajectory(robot,milestones = f)
# linkTraj = robotTraj.getLinkTrajectory(2,dt)
tj2  = tj.discretize(dt = 1)
len(tj2.milestones)

In [ ]:
milestones = np.array(tj2.milestones)
milestones = milestones[:,:2]

In [ ]:
milestones

In [ ]:
np.save('./towerbot_milestones.npy',milestones)

In [ ]:
from klampt import Geometry3D

In [ ]:
help(Geometry3D)